<a href="https://colab.research.google.com/github/leogalbu/Colab_Notebook/blob/NLP/NLP_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Check for GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-c58a2bc9-a871-6db2-8008-6d5b7006bd84)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import functions
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-09-27 12:20:03--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-27 12:20:04 (91.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset is Kaggle's introduction to NLP dataset (Tweets of disaster or not disaster)

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data('nlp_getting_started.zip')

--2022-09-27 12:20:06--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.4.128, 172.217.194.128, 142.251.10.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.4.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2022-09-27 12:20:06 (101 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualize the dataset

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# Chek test df
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class
train_df_shuffled['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Visualize random training examples
import random
# For df[[colname(s)]], the interior brackets are for list, and the outside brackets are indexing operator, i.e. 
# you must use double brackets if you select two or more columns. 
# With one column name, single pair of brackets returns a Series, while double brackets return a dataframe.
random_index = random.randint(0, len(train_df) - 5) # create random indexes, -5 for not overcome max len
for row in train_df_shuffled[['text', 'target']][random_index:random_index + 5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")


Target: 1 (real disaster)
Text:
16 dead in Russia bus accident: At least 16 people were killed and 26 others injured when two buses collided i... http://t.co/ybyP68ieVn

---

Target: 1 (real disaster)
Text:
#BHRAMABULL Watch Run The Jewels Use Facts to Defend Rioting in Ferguson: The socially minded duo takes on the... http://t.co/Ld5P1sIa2N

---

Target: 1 (real disaster)
Text:
I visited Hiroshima in 2006. It is an incredible place. This model shows devastation of the bomb. http://t.co/Gid6jqN8UG

---

Target: 1 (real disaster)
Text:
I scored 111020 points in PUNCH QUEST stopped when a squeaky bat collided into my skull. http://t.co/aEtgbxm1pL

---

Target: 1 (real disaster)
Text:
Someone teaching you that obedience will obliterate trials in your life is trying to sell you a used car. Jesus's life blows that theory.'

---



In [11]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), 
                                                                            train_df_shuffled['target'].to_numpy(), 
                                                                            test_size=0.1, 
                                                                            random_state=42)

In [13]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

With text problem, one of the first thing to do is to convert text to numbers. There are few ways to do this:
* Tokenization
* Embedding Dense vector of each token

In [18]:
# Tensorflow text vectorization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [20]:
# Use default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # How many words in the vocabulary,
                                    standardize='lower_and_strip_punctuation', # convert text to lower case and remove punctuation
                                    split="whitespace",
                                    ngrams=None, # Groups of N words
                                    output_mode='int', # how to map tokens to number
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False
                                    )

In [22]:
# Find the average number of tokens (words) in the training
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [25]:
# Setup text vectorization variables
max_vocab_length = 10000 # Max number of words to have in the vocabulary
max_length = 15 # max length out sequences will be (How many words from a tweet the model see) Prime 15 parole del tweet (media calcolata sopra)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length
                                    )

In [26]:
# Fit the text vectorizer to the training
text_vectorizer.adapt(train_sentences)

In [27]:
# Create a sample sentence and tokenize it
sample_sentence = "There's flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [30]:
# Choose a random sample from the training and tokenize it
random_sentence  = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\n\n Vectorized version:" ) 
text_vectorizer([random_sentence])

Original Text:
 http://t.co/X5XUMtoEkE Nuclear Emergency Current USA radiation levels monitoring site

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  105,   73, 3075,  799, 1375, 2065, 5098,  615,    0,    0,
           0,    0,    0,    0]])>

In [33]:
# Get the unique words in the vocabulary in the training
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] #get the most common words (first 5)
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Five most commmon words: {top_5_words}")
print(f"Five bottom common words: {bottom_5_words}")

Number of words in vocab: 10000
Five most commmon words: ['', '[UNK]', 'the', 'a', 'in']
Five bottom common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']
